In [1]:
from gridstatus import Ercot
import pandas as pd
ercot = Ercot()

In [2]:
def get_spred_df(year):
    # Fetch data (your code)
    df_rtm = ercot.get_rtm_spp(year=2023)  # ~800k rows, 15-min data
    df_dam = ercot.get_dam_spp(year=2023)  # ~1.3M rows, hourly data

    # Step 1: Inspect unique locations (pick one, e.g., 'HB_NORTH' for North Hub)
    print(df_rtm['Location'].unique())  # Outputs array of all settlement points
    location = 'HB_NORTH'  # Or 'LZ_NORTH', 'HB_BUSAVG', etc. – choose based on your strategy

    # Step 2: Filter to selected location
    df_rtm_loc = df_rtm[df_rtm['Location'] == location].copy()
    df_dam_loc = df_dam[df_dam['Location'] == location].copy()

    # Step 3: Parse datetimes (handles -06:00 offset automatically)
    df_rtm_loc['Interval Start'] = pd.to_datetime(df_rtm_loc['Interval Start'])
    df_dam_loc['Interval Start'] = pd.to_datetime(df_dam_loc['Interval Start'])

    # Step 4: Set index to 'Interval Start' for time-series ops
    df_rtm_loc = df_rtm_loc.set_index('Interval Start')
    df_dam_loc = df_dam_loc.set_index('Interval Start')

    # Step 5: Aggregate RTM to hourly (average 4x 15-min prices per hour)
    # Resample to 'H' (hourly), mean of 'Settlement Point Price'
    df_rtm_hourly = df_rtm_loc['SPP'].resample('H').mean().to_frame(name='rt_price')

    # Step 6: Select DA prices (already hourly)
    df_dam_hourly = df_dam_loc['SPP'].to_frame(name='da_price')

    # Step 7: Merge on hourly index (inner join for matching times)
    df_merged = df_dam_hourly.merge(df_rtm_hourly, left_index=True, right_index=True)

    # Step 8: Calculate spread
    df_merged['spread'] = df_merged['rt_price'] - df_merged['da_price']

    # Output: Save or use for model
    df_merged.to_csv(f'ercot_spreads_{year}_{location}.csv')
    print(df_merged.head())
    return df_merged

In [3]:
for year in range(2010,2026):
    get_spred_df(year)

2025-11-08 21:30:26 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13061&_1762605026
2025-11-08 21:30:26 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13061&_1762605026 with {}
2025-11-08 21:31:13 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605073
2025-11-08 21:31:13 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605073 with {}
2025-11-08 21:31:14 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-08 21:31:16 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605076
2025-11-08 21:31:16 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605076 with {}
2025-11-08 21:31:23 - INFO - Fetching document https://www.ercot.com/misapp/servlet

<StringArray>
[ 'HB_BUSAVG', 'HB_HOUSTON',   'LZ_RAYBN',    'LZ_WEST',  'HB_HUBAVG',
   'LZ_SOUTH',     'LZ_CPS',     'HB_PAN',   'HB_NORTH',   'LZ_NORTH',
 'LZ_HOUSTON',     'LZ_AEN',    'LZ_LCRA',   'HB_SOUTH',    'HB_WEST']
Length: 15, dtype: string
                           da_price  rt_price   spread
Interval Start                                        
2023-01-01 00:00:00-06:00     10.48   -2.1175 -12.5975
2023-01-01 01:00:00-06:00      9.89   -1.1500 -11.0400
2023-01-01 02:00:00-06:00      8.49   -1.1700  -9.6600
2023-01-01 03:00:00-06:00      7.99   -0.0175  -8.0075
2023-01-01 04:00:00-06:00      8.13   -0.8125  -8.9425


2025-11-08 21:32:12 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605132
2025-11-08 21:32:12 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605132 with {}
2025-11-08 21:32:13 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-08 21:32:14 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605134
2025-11-08 21:32:14 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605134 with {}
2025-11-08 21:32:20 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605140
2025-11-08 21:32:20 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605140 with {}
2025-11-08 21:32:21 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

<StringArray>
[ 'HB_BUSAVG', 'HB_HOUSTON',   'LZ_RAYBN',    'LZ_WEST',  'HB_HUBAVG',
   'LZ_SOUTH',     'LZ_CPS',     'HB_PAN',   'HB_NORTH',   'LZ_NORTH',
 'LZ_HOUSTON',     'LZ_AEN',    'LZ_LCRA',   'HB_SOUTH',    'HB_WEST']
Length: 15, dtype: string
                           da_price  rt_price   spread
Interval Start                                        
2023-01-01 00:00:00-06:00     10.48   -2.1175 -12.5975
2023-01-01 01:00:00-06:00      9.89   -1.1500 -11.0400
2023-01-01 02:00:00-06:00      8.49   -1.1700  -9.6600
2023-01-01 03:00:00-06:00      7.99   -0.0175  -8.0075
2023-01-01 04:00:00-06:00      8.13   -0.8125  -8.9425


2025-11-08 21:33:10 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605190
2025-11-08 21:33:10 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605190 with {}
2025-11-08 21:33:11 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-08 21:33:12 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605192
2025-11-08 21:33:12 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605192 with {}
2025-11-08 21:33:19 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605199
2025-11-08 21:33:19 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605199 with {}
2025-11-08 21:33:20 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

...

2025-11-08 21:44:33 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605873
2025-11-08 21:44:33 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605873 with {}
2025-11-08 21:44:34 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-08 21:44:35 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605875
2025-11-08 21:44:35 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605875 with {}
2025-11-08 21:44:42 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605882
2025-11-08 21:44:42 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605882 with {}
2025-11-08 21:44:42 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

<StringArray>
[ 'HB_BUSAVG', 'HB_HOUSTON',   'LZ_RAYBN',    'LZ_WEST',  'HB_HUBAVG',
   'LZ_SOUTH',     'LZ_CPS',     'HB_PAN',   'HB_NORTH',   'LZ_NORTH',
 'LZ_HOUSTON',     'LZ_AEN',    'LZ_LCRA',   'HB_SOUTH',    'HB_WEST']
Length: 15, dtype: string
                           da_price  rt_price   spread
Interval Start                                        
2023-01-01 00:00:00-06:00     10.48   -2.1175 -12.5975
2023-01-01 01:00:00-06:00      9.89   -1.1500 -11.0400
2023-01-01 02:00:00-06:00      8.49   -1.1700  -9.6600
2023-01-01 03:00:00-06:00      7.99   -0.0175  -8.0075
2023-01-01 04:00:00-06:00      8.13   -0.8125  -8.9425


2025-11-08 21:45:29 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605929
2025-11-08 21:45:29 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605929 with {}
2025-11-08 21:45:30 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-08 21:45:32 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605932
2025-11-08 21:45:32 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762605932 with {}
2025-11-08 21:45:37 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605937
2025-11-08 21:45:37 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762605937 with {}
2025-11-08 21:45:38 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

<StringArray>
[ 'HB_BUSAVG', 'HB_HOUSTON',   'LZ_RAYBN',    'LZ_WEST',  'HB_HUBAVG',
   'LZ_SOUTH',     'LZ_CPS',     'HB_PAN',   'HB_NORTH',   'LZ_NORTH',
 'LZ_HOUSTON',     'LZ_AEN',    'LZ_LCRA',   'HB_SOUTH',    'HB_WEST']
Length: 15, dtype: string
                           da_price  rt_price   spread
Interval Start                                        
2023-01-01 00:00:00-06:00     10.48   -2.1175 -12.5975
2023-01-01 01:00:00-06:00      9.89   -1.1500 -11.0400
2023-01-01 02:00:00-06:00      8.49   -1.1700  -9.6600
2023-01-01 03:00:00-06:00      7.99   -0.0175  -8.0075
2023-01-01 04:00:00-06:00      8.13   -0.8125  -8.9425


C:\Users\qwane\AppData\Local\Temp\ipykernel_27064\269279815.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_rtm_hourly = df_rtm_loc['SPP'].resample('H').mean().to_frame(name='rt_price')
